   # Perceptron Example

### Heaviside function with a default epsilon

In [ ]:
g <- function(net, epsilon=0.5) {
	if (net > epsilon) {
		return (1)
	} else {
		return (0)
	}
}

### Perceptron Training
This is the function that trains the Perceptron model. Observe the ETA and Threshold parameters assume default values 

In [ ]:
perceptron.train <- function(train.table, eta=0.1, threshold=1e-2) {
	nVars = ncol(train.table)-1 # Number of input variables
	cat("Randomizing weights and theta in range [-0.5, 0.5]...\n")
	weights = runif(min=-0.5, max=0.5, n=nVars) # Randomizing weights
	theta = runif(min=-0.5, max=0.5, n=1) # Randomizing theta

	# This sum of squared errors will accumulate all errors
	# occuring along training iterations. When this error is
	# below a given threshold, learning stops.
	sumSquaredError = 2*threshold

	# Learning iterations, this will occur until the classifier is
    # done with its parameters optimization given a threshold
	while (sumSquaredError > threshold) {

		# Initializing the sum of squared errors as zero
		# to start counting and later evaluate the total
		# loss for this dataset in train.table
		sumSquaredError = 0

		# Iterate along all rows (examples) contained in
		# train.table
		for (i in 1:nrow(train.table)) {

			# Example x_i
			x_i = train.table[i, 1:nVars]

			# Expected output class
			# Observe the last column of this table
			# contains the output class
			y_i = train.table[i, ncol(train.table)]

			# Now the Perceptron produces the output
			# class using the current values for 
			# weights and theta, then it applies the
			# heaviside function
			hat_y_i = g(x_i %*% weights + theta)

			# This is the error, referred to as (y_i - g(x_i))
			# in the Perceptron formulation
			Error = y_i - hat_y_i

			# As part of the Gradient Descent method, we here
			# compute the partial derivative of the Squared Error
			# for the current example i in terms of weights and theta.
			# Observe constant 2 is not necessary, once we
			# can set eta using the value we desire
			dE2_dw1 = 2 * Error * -x_i
			dE2_dtheta = 2 * Error * -1

			# This is the Gradient Descent method to adapt
			# weights and theta as defined in the formulation
			weights = weights - eta * dE2_dw1
			theta = theta - eta * dE2_dtheta

			# Accumulating the squared error to define the stop criterion
			sumSquaredError = sumSquaredError + Error^2
		}

		cat("Accumulated sum of squared errors = ", sumSquaredError, "\n")
	}

	# Returning weights and theta, once they represent the solution
	ret = list()
	ret$weights = weights
	ret$theta = theta

	return (ret)
}

### This is the function to execute the Perceptron over unseen data (new examples)

In [ ]:
perceptron.test <- function(test.table, weights, theta) {

	# Here we print out the expected class (yi) followed by the
	# obtained one (hat_yi) when considering weights and theta.
	# Of course, function perceptron.train should be called
	# previously, in order to find the values for weights and theta
	cat("#yi\that_yi\n")

	# Number of input variables
	nVars = ncol(test.table)-1

	# For every row in the test.table
	for (i in 1:nrow(test.table)) {

		# Example i
		x_i = test.table[i, 1:nVars]

		# Expected class for example i
		y_i = test.table[i, ncol(test.table)]

		# Output class produced by the Perceptron
		hat_y_i = g(x_i %*% weights + theta)

		cat(y_i, "\t", hat_y_i, "\n")
	}
}


In [ ]:
perceptron.run.simple <- function() {
	# This is a table with training examples
	train.table = matrix(c(0.0, 0,
			       0.1, 0,
			       0.2, 0,
			       0.3, 0,
			       0.4, 0,
			       0.5, 0,
			       0.6, 1,
			       0.7, 1,
			       0.8, 1,
			       0.9, 1,
			       1.0, 1),
			      nrow=11,
			      ncol=2,
			      byrow=TRUE)
	# This is a table with test examples.
	# The last column only shows the expected
	# output and it is not used in the testing stage
	test.table = matrix(c(0.05, 0,
			      0.15, 0,
			      0.25, 0,
			      0.35, 0,
			      0.45, 0,
			      0.55, 1,
			      0.65, 1,
			      0.75, 1,
			      0.85, 1,
			      0.95, 1),
			     nrow=10,
			     ncol=2,
			     byrow=TRUE)

	# Training the Perceptron to find weights and theta
	training.result = perceptron.train(train.table)

	# Testing the Perceptron with the weights and theta found
	perceptron.test(test.table, training.result$weights, training.result$theta)

	return (training.result)
}

This function plots the hyperplane found for this simplest problem which considers a single input variable. Variables range.start and range.end define the interval of values for the single input variable composing the problem. This simple problem has a single variable composing every example i, which is x_i,1

In [ ]:
perceptron.simple.hyperplane.plot <- function(weight, theta, range.start=0, range.end=1) {

	# Number of variables is 1
	nVars = 1

	# We will now define the same range for the input variable.
	# This range will contain 100 discretized values
	range_of_every_input_variable = seq(range.start, range.end, length=100)
	x_1 = range_of_every_input_variable
	
	# Computing net for every input value of variable x_i,1
	all_nets = cbind(x_1, 1) %*% c(weight, theta)

	# This variable all_nets contains all net values for all values assumed
	# by variable x_1. Variable hat_y will contain the Perceptron outputs after
	# applying the heaviside function
	hat_y = rep(0, length(all_nets))
	for (i in 1:length(all_nets)) {
		hat_y[i] = g(all_nets[i])
	}

	# Variable hyperplane will contain two columns, the first corresponds to
	# the input value of x_i,1 and the second to the class produced by the
	# Perceptron
	hyperplane = cbind(x_1, hat_y)

	# Plotting the hyperplane found by the Perceptron
	#plot(hyperplane)
   
   plot(x_1, hat_y, cex.axis=1.25, cex=1.3, xlab="input x_1", ylab="output hat_y (classes) and hyperplane (w,theta)", cex.lab=1.25, yaxt="n", ylim=c(-0.1,1.1))
   lines(hyperplane, lwd=2.3, col="gray40",lty=2)
   axis(2, at=c(0,1), labels=c(0,1), cex.axis=1.25,cex.lab=1.25)

	return (hyperplane)
}

In [ ]:
hyperplane <- perceptron.run.simple()

perceptron.simple.hyperplane.plot(hyperplane$weights, hyperplane$theta)

# Perceptron Hyperplane without G

This function plots the hyperplane found for this simplest problem which considers a single input variable and  function net only. Variables range.start and range.end define the interval of values for the single input variable composing the problem. This simple problem has a single variable composing every example i, which is x_i,1

In [ ]:
perceptron.simple.hyperplane.plot.without.g <- function(weight, theta, range.start=0, range.end=1) {

	# Number of variables is 1
	nVars = 1

	# We will now define the same range for the input variable.
	# This range will contain 100 discretized values
	range_of_every_input_variable = seq(range.start, range.end, length=100)
	x_1 = range_of_every_input_variable
	
	# Computing net for every input value of variable x_i,1
	all_nets = cbind(x_1, 1) %*% c(weight, theta)

	# This variable all_nets contains all net values for every value assumed
	# by variable x_1. Variable hat_y will contain the Perceptron outputs before
	# applying the heaviside function
	hat_y = rep(0, length(all_nets))
	for (i in 1:length(all_nets)) {
		hat_y[i] = all_nets[i] # Without the heaviside function g(net)
	}

	# Variable hyperplane will contain two columns, the first corresponds to
	# the input value of x_i,1 and the second to the class produced by the
	# Perceptron
	hyperplane = cbind(x_1, hat_y)

	# Plotting the hyperplane found by the Perceptron in terms of function net
	plot(hyperplane)

	return (hyperplane)
}

## Running the Perceptron Hyperplane without G

This function is used to run the training stage for the simplest classification task. Each training will produce a different pair of weight and theta, which are then used to plot function net

In [ ]:
run.several.times <- function(times=5) {

	# Saving the results for each function net
	net.functions = list()

	# For each execution
	for (i in 1:times) {
		# Call training
		training.execution = perceptron.run.simple()

		# Obtaining function net
		net.functions[[i]] = perceptron.simple.hyperplane.plot.without.g(
					training.execution$weight, 
					training.execution$theta)
	}

	# Plotting
	plot(net.functions[[1]], col=1)
	for (i in 2:times) {
		points(net.functions[[i]], col=i)
	}
}

# Perceptron AND

In [10]:
perceptron.run.AND <- function() {

	# This is a table with all possible examples
	# for the problem AND. In this case we will
	# use the same table for training and testing,
	# just because we know all possible binary
	# combinations
	table = matrix(c(0, 0, 0,  # 0 AND 0 = 0
			 0, 1, 0,  # 0 AND 1 = 0
			 1, 0, 0,  # 1 AND 0 = 0
			 1, 1, 1), # 1 AND 1 = 1
			nrow=4,
			ncol=3,
			byrow=TRUE)

	# Training the Perceptron to find weights and theta
	training.result = perceptron.train(table)

	# Testing the Perceptron with the weights and theta found
	perceptron.test(table, training.result$weights, training.result$theta)

	# Plotting the hyperplane found
	perceptron.hyperplane.plot(training.result$weights, training.result$theta)
}

This function plots the hyperplane found for a given classification task with two input variables only. Variables range.start and range.end define the interval of values for every one of those input variables composing the problem. The problem AND has two variables composing each example i, which are x_i,1 and x_i,2

In [11]:
perceptron.hyperplane.plot <- function(weights, theta, range.start=0, range.end=1) {

	# Variable weights define the number of input variables we have,
	# so we can use this information to create axes in a multidimensional
	# input space in order to see how inputs modify the output class
	# provided by the Perceptron
	nVars = length(weights)

	# We will now define the same range for every input variable.
	# This range will contain 100 discretized values
	range_of_every_input_variable = seq(range.start, range.end, length=100)

	x_1 = range_of_every_input_variable
	x_2 = range_of_every_input_variable
	
	# Function outer combines every possible value for variable x_1
	# against every possible value for x_2. Observe they are continuous values
	# which were never seen (we expect either 0 or 1) by this Perceptron during 
	# the training stage. Also observe value 1 inside the cbind, which refers
	# to the 1 * theta while computing function net. Operation %*% corresponds
	# to the dot product.
	all_nets = outer(x_1, x_2, function(x, y) { cbind(x, y, 1) %*% c(weights, theta) } )

	# This variable all_nets contains all net values for every combination between
	# variables x_1 and x_2. Variable y will contain the Perceptron outputs after
	# applying the heaviside function
	y = matrix(0, nrow=nrow(all_nets), ncol=ncol(all_nets))
	for (row in 1:nrow(all_nets)) {
		for (col in 1:ncol(all_nets)) {
			y[row, col] = g(all_nets[row, col])
		}
	}

	# Plotting the hyperplane found by the Perceptron
	filled.contour(x_1, x_2, y)
}

# Calculating Perceptron's error rate

In [12]:
perceptron.simple.error <- function(range.start=-1, range.end=1, mu=1e-10) {

	# Defining the table with examples
	table = cbind(seq(0, 0.5, length=100), rep(0, 100)) # negative examples
	table = rbind(table, cbind(seq(0.5+mu, 1, length=100), rep(1, 100))) # positive examples

	# We will now define the same range for the free variables weight and theta.
	# This range will contain 100 discretized values
	range_for_free_variables = seq(range.start, range.end, length=100)
	weight = range_for_free_variables
	theta = range_for_free_variables
	
	# Sum of errors while varying weight and theta
	error_function = matrix(0, nrow=length(weight), ncol=length(theta))

	# For each weight
	for (w in 1:length(weight)) {
		# For each theta
		for (t in 1:length(theta)) {
			# Compute all net values
			net = cbind(table[,1], rep(1, nrow(table))) %*% c(weight[w], theta[t])
			# Defining a vector to save the Perceptron outputs
			hat_y = rep(0, length(net))
			# Producing the output classes
			for (n in 1:length(net)) {
				# We removed function g(net) to improve illustration
				hat_y[n] = net[n] 
			}

			# These are the expected classes
			y = table[,2]
			# Expected minus the obtained classes to provide the error
			error = y - hat_y
			# Saving the total error in the matrix
			error_function[w, t] = sum(error) # This brings up amortization problems
							  # because positive and negative terms
							  # will cancel each other
		}
	}

	# Plotting the error
	filled.contour(error_function)
}

In [13]:
perceptron.simple.squared.error <- function(range.start=-10, range.end=10, mu=1e-10) {

	# Defining the table with examples
	table = cbind(seq(0, 0.5, length=100), rep(0, 100))
	table = rbind(table, cbind(seq(0.5+mu, 1, length=100), rep(1, 100)))

	# We will now define the same range for the free variables weight and theta.
	# This range will contain 100 discretized values
	range_for_free_variables = seq(range.start, range.end, length=50)
	weight = range_for_free_variables
	theta = range_for_free_variables
	
	# Sum of squared errors while varying weight and theta
	error_function = matrix(0, nrow=length(weight), ncol=length(theta))

	# For each weight
	for (w in 1:length(weight)) {
		# For each theta
		for (t in 1:length(theta)) {
			# Compute all net values
			net = cbind(table[,1], rep(1, nrow(table))) %*% c(weight[w], theta[t])
			# Defining a vector to save the Perceptron outputs
			hat_y = rep(0, length(net))
			# Producing the output classes
			for (n in 1:length(net)) {
				# We removed function g(net) to improve illustration
				hat_y[n] = net[n]
			}

			# These are the expected classes
			y = table[,2]
			# Expected minus the obtained classes to provide the error,
			# which is then squared to avoid negative and positive values
			# that amortize each other
			squared.error = (y - hat_y)^2
			# Saving the total squared error in the matrix
			error_function[w, t] = sum(squared.error)
		}
	}

	# We apply a log on the squared error function to improve illustration,
	# otherwise we do not see the paraboloid as clear as in this form.
	filled.contour(log(error_function))
}
